In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import time

term = "natural product"
start_year = 2000
end_year = 2007
csv_file = "pubmed_alkaloid_results.csv"
columns = ["Period", "PMID", "DOI", "Abstract"]  # 修改列名以包含时间段信息

# 初始化CSV文件
pd.DataFrame(columns=columns).to_csv(csv_file, index=False)


def get_literature_id(term, start_date, end_date):
    """获取指定日期范围的文献ID列表"""
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    params = {
        "db": "pubmed",
        "term": f"{term} AND {start_date}:{end_date}[pdat]",
        "retmax": 100000
    }

    try:
        time.sleep(0.34)  # 遵守NCBI速率限制
        response = requests.get(base_url, params=params, timeout=10)
        response.raise_for_status()
    except Exception as e:
        print(f"Error fetching IDs for {start_date}-{end_date}: {str(e)}")
        return [], 0

    soup = BeautifulSoup(response.text, "xml")
    count = int(soup.find("Count").text) if soup.find("Count") else 0
    id_list = [id_tag.text for id_tag in soup.find_all("Id")]

    return id_list, count


def parse_article(article_text):
    """稳健的文献解析函数"""
    record = {"PMID": None, "DOI": None, "Abstract": None}

    try:
        lines = [line.strip() for line in article_text.split('\n')]

        # PMID提取
        pmid_line = next(line for line in lines if line.startswith("PMID- "))
        record["PMID"] = pmid_line.split("- ")[1]

        # DOI提取
        for line in lines:
            if line.startswith("AID - ") and "[doi]" in line:
                record["DOI"] = line.split("- ")[1].split("[doi]")[0].strip()
                break

        # 摘要提取
        abstract = []
        in_abstract = False
        for line in lines:
            if line.startswith("AB  - "):
                in_abstract = True
                abstract.append(line[6:].strip())
            elif in_abstract:
                if line and not line.startswith(("PMID-", "AID -", "FAU -")):
                    abstract.append(line)
                else:
                    break
        record["Abstract"] = " ".join(abstract) if abstract else None

    except Exception as e:
        print(f"解析文献时发生错误: {str(e)}")

    return record


def get_detailed_info(id_list, period):
    """获取文献详细信息并包含时间段"""
    extracted_info = []
    if not id_list:
        return extracted_info

    for i in range(0, len(id_list), 200):
        batch_ids = id_list[i:i + 200]
        params = {
            "db": "pubmed",
            "id": ",".join(batch_ids),
            "rettype": "medline",
            "retmode": "text"
        }

        max_retries = 3
        
        for i in range(max_retries):
            try:
                time.sleep(0.34)
                response = requests.get(
                    "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi",
                    params=params,
                    timeout=10
                )
                response.raise_for_status()
                break  # 请求成功，退出循环
            except Exception as e:
                if i == max_retries - 1:  # 如果是最后一次重试失败
                    print(f"Error fetching details for {period} batch {i}: {str(e)}")
                else:
                    print(f"Retrying... Attempt {i + 1}/{max_retries}")
                    continue
        records = re.findall(r'PMID- .*?(?=\nPMID- |\Z)', response.text, flags=re.DOTALL)

        for record_text in records:
            parsed = parse_article(record_text)
            if not parsed["PMID"]:
                print(f"跳过无法解析PMID的记录，时间段：{period}")
                continue
            extracted_info.append([
                period,
                parsed["PMID"],
                parsed["DOI"],
                parsed["Abstract"]
            ])

    return extracted_info


# 主循环处理每个半年
for year in range(start_year, end_year + 1):
    # 上半年（H1）
    h1_start = f"{year}/01/01"
    h1_end = f"{year}/06/30"
    h1_period = f"{year}H1"
    print(f"Processing {h1_period}...")

    h1_ids, h1_count = get_literature_id(term, h1_start, h1_end)
    print(f"Found {h1_count} articles in {h1_period}")

    if h1_count > 0:
        h1_details = get_detailed_info(h1_ids, h1_period)
        if h1_details:
            pd.DataFrame(h1_details, columns=columns).to_csv(
                csv_file, mode="a", header=False, index=False
            )
            print(f"Saved {len(h1_details)} records for {h1_period}\n")

    # 下半年（H2）
    h2_start = f"{year}/07/01"
    h2_end = f"{year}/12/31"
    h2_period = f"{year}H2"
    print(f"Processing {h2_period}...")

    h2_ids, h2_count = get_literature_id(term, h2_start, h2_end)
    print(f"Found {h2_count} articles in {h2_period}")

    if h2_count > 0:
        h2_details = get_detailed_info(h2_ids, h2_period)
        if h2_details:
            pd.DataFrame(h2_details, columns=columns).to_csv(
                csv_file, mode="a", header=False, index=False
            )
            print(f"Saved {len(h2_details)} records for {h2_period}\n")

print("所有时间段处理完成！")

Processing 2000H1...
Found 5784 articles in 2000H1
Saved 5784 records for 2000H1

Processing 2000H2...
Found 5161 articles in 2000H2
Saved 5161 records for 2000H2

Processing 2001H1...
Found 6457 articles in 2001H1
Saved 6457 records for 2001H1

Processing 2001H2...
Found 5676 articles in 2001H2
Saved 5676 records for 2001H2

Processing 2002H1...
Found 7121 articles in 2002H1
Saved 7121 records for 2002H1

Processing 2002H2...
Found 6184 articles in 2002H2
Retrying... Attempt 1/3
Saved 6184 records for 2002H2

Processing 2003H1...
Found 8080 articles in 2003H1
Saved 8080 records for 2003H1

Processing 2003H2...
Found 6690 articles in 2003H2
Saved 6690 records for 2003H2

Processing 2004H1...
Found 8180 articles in 2004H1
Saved 8180 records for 2004H1

Processing 2004H2...
Found 7441 articles in 2004H2
Saved 7441 records for 2004H2

Processing 2005H1...
Found 9143 articles in 2005H1
Retrying... Attempt 1/3
Saved 9143 records for 2005H1

Processing 2005H2...
Found 8497 articles in 2005H2

In [2]:
#每四个月处理一次
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import time
from datetime import datetime, timedelta

term = "natural product"
start_year = 2000
end_year = 2007
csv_file = "pubmed_np_results2.csv"
columns = ["Period", "PMID", "DOI", "Abstract"]  # 修改列名以包含时间段信息

# 初始化CSV文件
pd.DataFrame(columns=columns).to_csv(csv_file, index=False)


def get_literature_id(term, start_date, end_date):
    """获取指定日期范围的文献ID列表"""
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    params = {
        "db": "pubmed",
        "term": f"{term} AND {start_date}:{end_date}[pdat]",
        "retmax": 100000
    }

    try:
        time.sleep(0.34)  # 遵守NCBI速率限制
        response = requests.get(base_url, params=params, timeout=10)
        response.raise_for_status()
    except Exception as e:
        print(f"Error fetching IDs for {start_date}-{end_date}: {str(e)}")
        return [], 0

    soup = BeautifulSoup(response.text, "xml")
    count = int(soup.find("Count").text) if soup.find("Count") else 0
    id_list = [id_tag.text for id_tag in soup.find_all("Id")]

    return id_list, count


def parse_article(article_text):
    """稳健的文献解析函数"""
    record = {"PMID": None, "DOI": None, "Abstract": None}

    try:
        lines = [line.strip() for line in article_text.split('\n')]

        # PMID提取
        pmid_line = next(line for line in lines if line.startswith("PMID- "))
        record["PMID"] = pmid_line.split("- ")[1]

        # DOI提取
        for line in lines:
            if line.startswith("AID - ") and "[doi]" in line:
                record["DOI"] = line.split("- ")[1].split("[doi]")[0].strip()
                break

        # 摘要提取
        abstract = []
        in_abstract = False
        for line in lines:
            if line.startswith("AB  - "):
                in_abstract = True
                abstract.append(line[6:].strip())
            elif in_abstract:
                if line and not line.startswith(("PMID-", "AID -", "FAU -")):
                    abstract.append(line)
                else:
                    break
        record["Abstract"] = " ".join(abstract) if abstract else None

    except Exception as e:
        print(f"解析文献时发生错误: {str(e)}")

    return record


def get_detailed_info(id_list, period):
    """获取文献详细信息并包含时间段"""
    extracted_info = []
    if not id_list:
        return extracted_info

    for i in range(0, len(id_list), 200):
        batch_ids = id_list[i:i + 200]
        params = {
            "db": "pubmed",
            "id": ",".join(batch_ids),
            "rettype": "medline",
            "retmode": "text"
        }

        max_retries = 3
        
        for i in range(max_retries):
            try:
                time.sleep(0.34)
                response = requests.get(
                    "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi",
                    params=params,
                    timeout=10
                )
                response.raise_for_status()
                break  # 请求成功，退出循环
            except Exception as e:
                if i == max_retries - 1:  # 如果是最后一次重试失败
                    print(f"Error fetching details for {period} batch {i}: {str(e)}")
                else:
                    print(f"Retrying... Attempt {i + 1}/{max_retries}")
                    continue
        records = re.findall(r'PMID- .*?(?=\nPMID- |\Z)', response.text, flags=re.DOTALL)

        for record_text in records:
            parsed = parse_article(record_text)
            if not parsed["PMID"]:
                print(f"跳过无法解析PMID的记录，时间段：{period}")
                continue
            extracted_info.append([
                period,
                parsed["PMID"],
                parsed["DOI"],
                parsed["Abstract"]
            ])

    return extracted_info
    
# 主循环处理每四个月
for year in range(start_year, end_year + 1):
    # 每四个月的开始月份
    start_months = [1, 5, 9]
    
    for i, start_month in enumerate(start_months):
        # 计算时间段的开始和结束日期
        start_date = datetime(year, start_month, 1)
        
        # 如果是最后一个时间段（9-12月），结束日期为当年12月31日
        if i == len(start_months) - 1:
            end_date = datetime(year, 12, 31)
        else:
            # 下一个时间段的开始日期减去一天
            next_start_date = datetime(year, start_months[i+1], 1)
            end_date = next_start_date - timedelta(days=1)
        
        # 格式化日期为字符串
        period_start = start_date.strftime("%Y/%m/%d")
        period_end = end_date.strftime("%Y/%m/%d")
        period_name = f"{year}Q{i+1}"  # 例如：2023Q1, 2023Q2, 2023Q3
        
        print(f"Processing {period_name}...")
        
        # 获取文献ID和数量
        ids, count = get_literature_id(term, period_start, period_end)
        print(f"Found {count} articles in {period_name}")
        
        if count > 0:
            details = get_detailed_info(ids, period_name)
            if details:
                pd.DataFrame(details, columns=columns).to_csv(
                    csv_file, mode="a", header=False, index=False
                )
                print(f"Saved {len(details)} records for {period_name}\n")

print("所有时间段处理完成！")

Processing 2000Q1...
Found 4169 articles in 2000Q1
Saved 4169 records for 2000Q1

Processing 2000Q2...
Found 3208 articles in 2000Q2
Saved 3208 records for 2000Q2

Processing 2000Q3...
Found 3568 articles in 2000Q3
Saved 3568 records for 2000Q3

Processing 2001Q1...
Found 4631 articles in 2001Q1
Saved 4631 records for 2001Q1

Processing 2001Q2...
Found 3650 articles in 2001Q2
Saved 3650 records for 2001Q2

Processing 2001Q3...
Found 3891 articles in 2001Q3
Saved 3891 records for 2001Q3

Processing 2002Q1...
Found 4984 articles in 2002Q1
Saved 4984 records for 2002Q1

Processing 2002Q2...
Found 4147 articles in 2002Q2
Saved 4147 records for 2002Q2

Processing 2002Q3...
Found 4211 articles in 2002Q3
Saved 4211 records for 2002Q3

Processing 2003Q1...
Found 5638 articles in 2003Q1
Saved 5638 records for 2003Q1

Processing 2003Q2...
Found 4633 articles in 2003Q2
Saved 4633 records for 2003Q2

Processing 2003Q3...
Found 4594 articles in 2003Q3
Saved 4594 records for 2003Q3

Processing 2004Q

In [3]:
#每1个月处理一次
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import time
from datetime import datetime, timedelta

term = "natural product"
start_year = 2008
end_year = 2020
csv_file = "pubmed_np_results3.csv"
columns = ["Period", "PMID", "DOI", "Abstract"]  # 修改列名以包含时间段信息

# 初始化CSV文件
pd.DataFrame(columns=columns).to_csv(csv_file, index=False)


def get_literature_id(term, start_date, end_date):
    """获取指定日期范围的文献ID列表"""
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    params = {
        "db": "pubmed",
        "term": f"{term} AND {start_date}:{end_date}[pdat]",
        "retmax": 100000
    }

    try:
        time.sleep(0.34)  # 遵守NCBI速率限制
        response = requests.get(base_url, params=params, timeout=10)
        response.raise_for_status()
    except Exception as e:
        print(f"Error fetching IDs for {start_date}-{end_date}: {str(e)}")
        return [], 0

    soup = BeautifulSoup(response.text, "xml")
    count = int(soup.find("Count").text) if soup.find("Count") else 0
    id_list = [id_tag.text for id_tag in soup.find_all("Id")]

    return id_list, count


def parse_article(article_text):
    """稳健的文献解析函数"""
    record = {"PMID": None, "DOI": None, "Abstract": None}

    try:
        lines = [line.strip() for line in article_text.split('\n')]

        # PMID提取
        pmid_line = next(line for line in lines if line.startswith("PMID- "))
        record["PMID"] = pmid_line.split("- ")[1]

        # DOI提取
        for line in lines:
            if line.startswith("AID - ") and "[doi]" in line:
                record["DOI"] = line.split("- ")[1].split("[doi]")[0].strip()
                break

        # 摘要提取
        abstract = []
        in_abstract = False
        for line in lines:
            if line.startswith("AB  - "):
                in_abstract = True
                abstract.append(line[6:].strip())
            elif in_abstract:
                if line and not line.startswith(("PMID-", "AID -", "FAU -")):
                    abstract.append(line)
                else:
                    break
        record["Abstract"] = " ".join(abstract) if abstract else None

    except Exception as e:
        print(f"解析文献时发生错误: {str(e)}")

    return record


def get_detailed_info(id_list, period):
    """获取文献详细信息并包含时间段"""
    extracted_info = []
    if not id_list:
        return extracted_info

    for i in range(0, len(id_list), 200):
        batch_ids = id_list[i:i + 200]
        params = {
            "db": "pubmed",
            "id": ",".join(batch_ids),
            "rettype": "medline",
            "retmode": "text"
        }

        max_retries = 3
        
        for i in range(max_retries):
            try:
                time.sleep(0.34)
                response = requests.get(
                    "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi",
                    params=params,
                    timeout=10
                )
                response.raise_for_status()
                break  # 请求成功，退出循环
            except Exception as e:
                if i == max_retries - 1:  # 如果是最后一次重试失败
                    print(f"Error fetching details for {period} batch {i}: {str(e)}")
                else:
                    print(f"Retrying... Attempt {i + 1}/{max_retries}")
                    continue
        records = re.findall(r'PMID- .*?(?=\nPMID- |\Z)', response.text, flags=re.DOTALL)

        for record_text in records:
            parsed = parse_article(record_text)
            if not parsed["PMID"]:
                print(f"跳过无法解析PMID的记录，时间段：{period}")
                continue
            extracted_info.append([
                period,
                parsed["PMID"],
                parsed["DOI"],
                parsed["Abstract"]
            ])

    return extracted_info
    
# 主循环处理每月
for year in range(start_year, end_year + 1):
    for month in range(1, 13):  # 遍历1到12月
        # 计算每个月的开始日期和结束日期
        start_date = datetime(year, month, 1)
        
        # 如果是12月，结束日期为当年12月31日
        if month == 12:
            end_date = datetime(year, 12, 31)
        else:
            # 下个月的开始日期减去一天
            next_month_start = datetime(year, month + 1, 1)
            end_date = next_month_start - timedelta(days=1)
        
        # 格式化日期为字符串
        period_start = start_date.strftime("%Y/%m/%d")
        period_end = end_date.strftime("%Y/%m/%d")
        period_name = f"{year}-{month:02d}"  # 例如：2023-01, 2023-02, ...
        
        print(f"Processing {period_name}...")
        
        # 获取文献ID和数量
        ids, count = get_literature_id(term, period_start, period_end)
        print(f"Found {count} articles in {period_name}")
        
        if count > 0:
            details = get_detailed_info(ids, period_name)
            if details:
                pd.DataFrame(details, columns=columns).to_csv(
                    csv_file, mode="a", header=False, index=False
                )
                print(f"Saved {len(details)} records for {period_name}\n")

print("所有时间段处理完成！")

Processing 2008-01...
Found 3369 articles in 2008-01
Saved 3369 records for 2008-01

Processing 2008-02...
Found 1939 articles in 2008-02
Saved 1939 records for 2008-02

Processing 2008-03...
Found 2016 articles in 2008-03
Saved 2016 records for 2008-03

Processing 2008-04...
Found 2024 articles in 2008-04
Saved 2024 records for 2008-04

Processing 2008-05...
Found 2010 articles in 2008-05
Saved 2010 records for 2008-05

Processing 2008-06...
Found 2062 articles in 2008-06
Saved 2062 records for 2008-06

Processing 2008-07...
Found 2110 articles in 2008-07
Saved 2110 records for 2008-07

Processing 2008-08...
Found 1995 articles in 2008-08
Saved 1995 records for 2008-08

Processing 2008-09...
Found 2186 articles in 2008-09
Saved 2186 records for 2008-09

Processing 2008-10...
Found 2050 articles in 2008-10
Saved 2050 records for 2008-10

Processing 2008-11...
Found 2082 articles in 2008-11
Saved 2082 records for 2008-11

Processing 2008-12...
Found 2134 articles in 2008-12
Saved 2134 r

In [4]:
#每半个月处理一次
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import time
from datetime import datetime, timedelta
from calendar import monthrange

term = "natural product"
start_year = 2021
end_year = 2025
csv_file = "pubmed_np_results5.csv"
columns = ["Period", "PMID", "DOI", "Abstract"]  # 修改列名以包含时间段信息

# 初始化CSV文件
pd.DataFrame(columns=columns).to_csv(csv_file, index=False)


def get_literature_id(term, start_date, end_date):
    """获取指定日期范围的文献ID列表"""
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    params = {
        "db": "pubmed",
        "term": f"{term} AND {start_date}:{end_date}[pdat]",
        "retmax": 100000
    }

    try:
        time.sleep(0.34)  # 遵守NCBI速率限制
        response = requests.get(base_url, params=params, timeout=10)
        response.raise_for_status()
    except Exception as e:
        print(f"Error fetching IDs for {start_date}-{end_date}: {str(e)}")
        return [], 0

    soup = BeautifulSoup(response.text, "xml")
    count = int(soup.find("Count").text) if soup.find("Count") else 0
    id_list = [id_tag.text for id_tag in soup.find_all("Id")]

    return id_list, count


def parse_article(article_text):
    """稳健的文献解析函数"""
    record = {"PMID": None, "DOI": None, "Abstract": None}

    try:
        lines = [line.strip() for line in article_text.split('\n')]

        # PMID提取
        pmid_line = next(line for line in lines if line.startswith("PMID- "))
        record["PMID"] = pmid_line.split("- ")[1]

        # DOI提取
        for line in lines:
            if line.startswith("AID - ") and "[doi]" in line:
                record["DOI"] = line.split("- ")[1].split("[doi]")[0].strip()
                break

        # 摘要提取
        abstract = []
        in_abstract = False
        for line in lines:
            if line.startswith("AB  - "):
                in_abstract = True
                abstract.append(line[6:].strip())
            elif in_abstract:
                if line and not line.startswith(("PMID-", "AID -", "FAU -")):
                    abstract.append(line)
                else:
                    break
        record["Abstract"] = " ".join(abstract) if abstract else None

    except Exception as e:
        print(f"解析文献时发生错误: {str(e)}")

    return record


def get_detailed_info(id_list, period):
    """获取文献详细信息并包含时间段"""
    extracted_info = []
    if not id_list:
        return extracted_info

    for i in range(0, len(id_list), 200):
        batch_ids = id_list[i:i + 200]
        params = {
            "db": "pubmed",
            "id": ",".join(batch_ids),
            "rettype": "medline",
            "retmode": "text"
        }

        max_retries = 3
        
        for i in range(max_retries):
            try:
                time.sleep(0.34)
                response = requests.get(
                    "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi",
                    params=params,
                    timeout=10
                )
                response.raise_for_status()
                break  # 请求成功，退出循环
            except Exception as e:
                if i == max_retries - 1:  # 如果是最后一次重试失败
                    print(f"Error fetching details for {period} batch {i}: {str(e)}")
                else:
                    print(f"Retrying... Attempt {i + 1}/{max_retries}")
                    continue
        records = re.findall(r'PMID- .*?(?=\nPMID- |\Z)', response.text, flags=re.DOTALL)

        for record_text in records:
            parsed = parse_article(record_text)
            if not parsed["PMID"]:
                print(f"跳过无法解析PMID的记录，时间段：{period}")
                continue
            extracted_info.append([
                period,
                parsed["PMID"],
                parsed["DOI"],
                parsed["Abstract"]
            ])

    return extracted_info
    
# 主循环处理每半个月
for year in range(start_year, end_year + 1):
    for month in range(1, 13):  # 遍历1到12月
        # 获取当月的天数
        days_in_month = monthrange(year, month)[1]
        
        # 第一个时间段：1-15日
        first_period_start = datetime(year, month, 1)
        first_period_end = datetime(year, month, 15)
        first_period_name = f"{year}-{month:02d}-上半月"
        
        print(f"Processing {first_period_name}...")
        
        # 获取文献ID和数量
        ids, count = get_literature_id(term, 
                                      first_period_start.strftime("%Y/%m/%d"), 
                                      first_period_end.strftime("%Y/%m/%d"))
        print(f"Found {count} articles in {first_period_name}")
        
        if count > 0:
            details = get_detailed_info(ids, first_period_name)
            if details:
                pd.DataFrame(details, columns=columns).to_csv(
                    csv_file, mode="a", header=False, index=False
                )
                print(f"Saved {len(details)} records for {first_period_name}\n")
        
        # 第二个时间段：16日-月底
        second_period_start = datetime(year, month, 16)
        second_period_end = datetime(year, month, days_in_month)
        second_period_name = f"{year}-{month:02d}-下半月"
        
        print(f"Processing {second_period_name}...")
        
        # 获取文献ID和数量
        ids, count = get_literature_id(term, 
                                      second_period_start.strftime("%Y/%m/%d"), 
                                      second_period_end.strftime("%Y/%m/%d"))
        print(f"Found {count} articles in {second_period_name}")
        
        if count > 0:
            details = get_detailed_info(ids, second_period_name)
            if details:
                pd.DataFrame(details, columns=columns).to_csv(
                    csv_file, mode="a", header=False, index=False
                )
                print(f"Saved {len(details)} records for {second_period_name}\n")

print("所有时间段处理完成！")

Processing 2021-01-上半月...
Found 7787 articles in 2021-01-上半月
Saved 7787 records for 2021-01-上半月

Processing 2021-01-下半月...
Found 1763 articles in 2021-01-下半月
Saved 1763 records for 2021-01-下半月

Processing 2021-02-上半月...
Found 3102 articles in 2021-02-上半月
Saved 3102 records for 2021-02-上半月

Processing 2021-02-下半月...
Found 1460 articles in 2021-02-下半月
Saved 1460 records for 2021-02-下半月

Processing 2021-03-上半月...
Found 3411 articles in 2021-03-上半月
Saved 3411 records for 2021-03-上半月

Processing 2021-03-下半月...
Found 1960 articles in 2021-03-下半月
Saved 1960 records for 2021-03-下半月

Processing 2021-04-上半月...
Found 2958 articles in 2021-04-上半月
Saved 2958 records for 2021-04-上半月

Processing 2021-04-下半月...
Found 1857 articles in 2021-04-下半月
Saved 1857 records for 2021-04-下半月

Processing 2021-05-上半月...
Found 3187 articles in 2021-05-上半月
Saved 3187 records for 2021-05-上半月

Processing 2021-05-下半月...
Found 1770 articles in 2021-05-下半月
Saved 1770 records for 2021-05-下半月

Processing 2021-06-上半月...
Foun